### Install LM-Critic

In [ ]:
!pip install torch==1.6.0 torchvision==0.7.0
!pip install transformers==4.3.3 datasets==1.3.0 absl-py rouge-score
!pip install nltk wandb editdistance spacy==3.0.5
!python3 -m nltk.downloader punkt

In [ ]:
from google.colab import drive, output
drive.mount('/content/drive/')

Mounted at /content/drive/


!unzip /content/drive/MyDrive/LM-Critic-main.zip -d /content/drive/MyDrive/Project/

In [ ]:
%cd /content/drive/MyDrive/Project/LM-Critic-main/

/content/drive/MyDrive/Project/LM-Critic-main


In [ ]:
%cd gec

/content/drive/MyDrive/Project/LM-Critic-main/gec


In [ ]:
f = open('data/round0_synthetic_scigpt/synthetic_50k.json','r')
d = [i for i in f]
f.close()
f = open('data/round0_synthetic_scigpt_10k/synthetic_10k.json','w')
[f.write(i) for i in d[:10000]]
f.close()
len(d)

50000

Test Critic

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python3 ../critic/critic_scigpt.py

CKP 0
CKP 1
Loaded ../scigpt/best-checkpoint
CKP 2
Enter a sentence: it is also expected to improve the process by managing the factors , ultimately achieving an improvement in construction productivity
Bad! Your sentence log(p) = -92.737
Neighbor sentence with highest log(p): , it is also expected to improve the process by managing the factors, ultimately achieving an improvement in construction productivity (= -91.835)
Enter a sentence: it is also expected to improve the process by managing the factors, ultimately achieving an improvement in construction productivity
Good! Your sentence log(p) = -92.737
Enter a sentence: Traceback (most recent call last):
  File "../critic/critic_scigpt.py", line 140, in <module>
KeyboardInterrupt


Eval Critic

In [ ]:
f = open('eval.json','r')
d = [i for i in f]
f.close()
len(d)

1949

In [ ]:
f = open('eval_ledat.json','w')
[f.write(i) for i in d[:600]]
f.close()

In [ ]:
%cd ../gec

/content/drive/MyDrive/Project/LM-Critic-main/gec


In [ ]:
!python ../eval_critic/eval_critic.py

CKP 0
CKP 1
Loaded ../scigpt/best-checkpoint
CKP 2
100% 20/20 [00:03<00:00,  6.51it/s]
100% 20/20 [00:02<00:00,  6.86it/s]
log p(bad) < log p(good)? 387 / 600 = 0.645

Baseline critic:
  Good precision = 344 / 683 = 0.504
  Good recall    = 344 / 600 = 0.573
  Good F0.5      = 0.516
  Bad precision  = 261 / 517 = 0.505
  Bad recall     = 261 / 600 = 0.435
  Bad F0.5       = 0.489
NO_SAMPLES 100
100% 600/600 [02:18<00:00,  4.34it/s]
100% 600/600 [02:14<00:00,  4.47it/s]

LM-Critic:
  Good precision = 345 / 660 = 0.523
  Good recall    = 345 / 600 = 0.575
  Good F0.5      = 0.532
  Bad precision  = 285 / 540 = 0.528
  Bad recall     = 285 / 600 = 0.475
  Bad F0.5       = 0.516


### Train Initial Fixer

In [ ]:
path = 'data/round2_scigpt'
gpt = 'scigpt'
gpt_cmd = '-scigpt True' # -scigpt True

In [ ]:
!mkdir -p {path}/fixer

In [ ]:
!python -u src/run_seq2seq.py \
    --model_name_or_path facebook/bart-base --task summarization --text_column bad_detoked --summary_column good_detoked \
    --do_train --num_train_epochs 1 --train_file {path}/round_train.json \
    --preprocessing_num_workers 20 --overwrite_output_dir --output_dir {path}/fixer --predict_with_generate --fp16 \
    --per_device_train_batch_size 64 --gradient_accumulation_steps 8 --max_source_length 64 --max_target_length 64 \
    --logging_first_step --logging_steps 20 --save_steps 2000 \
  |& tee {path}/fixer/log.txt

07/22/2022 13:08:47 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
07/22/2022 13:08:47 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='data/round2_scigpt/fixer', overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=64, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=8, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir='runs/Jul22_13-08-47_e02edc818058', logging_first_step=True, logging_steps=20, save_steps=2000, save_total_limit=None, no_cuda=False, seed=42, fp16=True, fp16_

Run Fixer on BEA-19

In [ ]:
!pip install spacy

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 6.8 MB/s 


In [ ]:
!python src/run_fixer.py -m {path}/fixer -i benchmarks/wi+locness_v2.1.bea19/m2/ABCN.dev.bea19.orig.txt -o {path}/predictions/bea19dev.out.txt --bea19

file data/round2_scigpt/fixer/config.json not found
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py", line 424, in get_config_dict
    use_auth_token=use_auth_token,
  File "/usr/local/lib/python3.7/dist-packages/transformers/file_utils.py", line 1093, in cached_path
    raise EnvironmentError("file {} not found".format(url_or_filename))
OSError: file data/round2_scigpt/fixer/config.json not found

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "src/run_fixer.py", line 22, in <module>
    model = BartForConditionalGeneration.from_pretrained(args.model_path, force_bos_token_to_be_generated=True)
  File "/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py", line 962, in from_pretrained
    **kwargs,
  File "/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py", line 376, in from_pretrained
    config_dict, kwargs = c

BEA T5

In [ ]:
!python src/run_fixer_t5.py -m {path}/fixer -i benchmarks/wi+locness_v2.1.bea19/m2/ABCN.dev.bea19.orig.txt -o {path}/predictions/bea19dev.out1.txt --bea19

T5
100% 110/110 [19:15<00:00, 10.51s/it]


CoNLL-14

In [ ]:
!python src/run_fixer.py -m {path}/fixer -i benchmarks/official-2014.combined.orig.txt -o {path}/predictions/conll.txt

file data/round2_scigpt/fixer/config.json not found
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py", line 424, in get_config_dict
    use_auth_token=use_auth_token,
  File "/usr/local/lib/python3.7/dist-packages/transformers/file_utils.py", line 1093, in cached_path
    raise EnvironmentError("file {} not found".format(url_or_filename))
OSError: file data/round2_scigpt/fixer/config.json not found

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "src/run_fixer.py", line 22, in <module>
    model = BartForConditionalGeneration.from_pretrained(args.model_path, force_bos_token_to_be_generated=True)
  File "/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py", line 962, in from_pretrained
    **kwargs,
  File "/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py", line 376, in from_pretrained
    config_dict, kwargs = c

Run Fixer on Ledat

In [ ]:
!python src/run_fixer.py -m {path}/fixer -i benchmarks/ledat.txt -o {path}/predictions/ledat.out.txt

file data/round2_scigpt/fixer/config.json not found
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py", line 424, in get_config_dict
    use_auth_token=use_auth_token,
  File "/usr/local/lib/python3.7/dist-packages/transformers/file_utils.py", line 1093, in cached_path
    raise EnvironmentError("file {} not found".format(url_or_filename))
OSError: file data/round2_scigpt/fixer/config.json not found

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "src/run_fixer.py", line 22, in <module>
    model = BartForConditionalGeneration.from_pretrained(args.model_path, force_bos_token_to_be_generated=True)
  File "/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py", line 962, in from_pretrained
    **kwargs,
  File "/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py", line 376, in from_pretrained
    config_dict, kwargs = c

Test Fixer

In [ ]:
!echo "This will prompt you for a sentence input, an returns the judgment (Good: grammatical, Bad: ungrammatical) along with the probability score of the input sentence." > test.txt

In [ ]:
!python src/run_fixer.py -m data/round0_synthetic/fixer -i test.txt -o test_out.txt
!cat test_out.txt

100% 1/1 [00:00<00:00,  1.46it/s]
mkdir: missing operand
Try 'mkdir --help' for more information.
this will prompt you for a sentence input , an returns the judgment ( good : grammatical , bad : ungrammatical ) along with the probability score of the input sentence


Run Critic on Data ( Get Training File for Fixer)

In [ ]:
%cd LM-Critic-main/gec

/content/drive/MyDrive/Project/LM-Critic-main/gec


In [ ]:
!python src/use_critic.py -f data/unlabelled_10k.txt -g data/unlabelled_scigpt/unlabelled_gd_10k.txt -b data/unlabelled_scigpt/unlabelled_bd_10k.txt -skip_indexes True -scigpt True

SCIGPT
CKP 0
CKP 1
Loaded ../scigpt/best-checkpoint
CKP 2
100% 10000/10000 [30:37<00:00,  5.44it/s]
src/use_critic.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save(f,np.array(res))
Good Sentences 2976
Bad Sentences 6992
Done


GPT on S2ORC:
Good Sentences 2960
Bad Sentences 7008

Run Fixer on unlabelled data

In [ ]:
!python src/run_fixer.py -m {path}/fixer -i data/unlabelled_{gpt}/unlabelled_bd_10k.txt -o {path}/data/fixer_output_unlabelled.txt

100% 175/175 [17:44<00:00,  6.08s/it]


In [ ]:
!python src/run_fixer.py -m lm-data/data/round1__BIFI/model-fixer/ -i data/unlabelled_yahoo/unlabelled_bd_10k.txt -o lm-data/data/round1__BIFI/fixer_output_unlabelled.txt

100% 149/149 [13:17<00:00,  5.35s/it]


Run Critic on Fixer Data

In [ ]:
!python src/use_critic.py -f {path}/data/fixer_output_unlabelled.txt -g {path}/data/fixer_output_unlabelled_gd.txt -b {path}/data/fixer_output_unlabelled_bd.txt {gpt_cmd}

SCIGPT
CKP 0
CKP 1
Loaded ../scigpt/best-checkpoint
CKP 2
100% 6992/6992 [22:22<00:00,  5.21it/s]
src/use_critic.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save(f,np.array(res))
Good Sentences 524
Bad Sentences 6468
Done


Gpt: Good Sentences 486
Bad Sentences 6522

Scigpt: Good Sentences 524
Bad Sentences 6468

Supervised gpt: Good Sentences 499
Bad Sentences 6509

supervised scigpt: Good Sentences 535
Bad Sentences 6457

Extract Training File

In [ ]:
!python src/extract_json.py -t {path}/data/fixer_output_unlabelled_gd.txt -p data/unlabelled_{gpt}/unlabelled_bd_10k.txt -o {path}/data/train_breaker.json -r {path}/data/train_fixer_A.json

Done


Train Breaker

In [ ]:
!mkdir -p {path}/breaker

In [ ]:
!python -u src/run_seq2seq.py \
    --model_name_or_path facebook/bart-base --task summarization --text_column bad_detoked --summary_column good_detoked \
    --do_train --num_train_epochs 1 --train_file {path}/data/train_breaker.json \
    --preprocessing_num_workers 20 --overwrite_output_dir --output_dir {path}/breaker --predict_with_generate --fp16 \
    --per_device_train_batch_size 64 --gradient_accumulation_steps 8 --max_source_length 64 --max_target_length 64 \
    --logging_first_step --logging_steps 20 --save_steps 2000 \
  |& tee {path}/breaker/log.txt

07/21/2022 15:11:30 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
07/21/2022 15:11:30 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='data/round1_scigpt/breaker', overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=64, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=8, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir='runs/Jul21_15-11-30_af59b7f1b83b', logging_first_step=True, logging_steps=20, save_steps=2000, save_total_limit=None, no_cuda=False, seed=42, fp16=True, fp1

Run Breaker on Data

In [ ]:
!python src/run_fixer.py -m {path}/breaker -i data/unlabelled_{gpt}/unlabelled_gd_10k.txt -o {path}/data/breaker_output_unlabelled.txt

100% 75/75 [07:01<00:00,  5.62s/it]


Run Critic on Breaker Data

In [ ]:
!python src/use_critic.py -f {path}/data/breaker_output_unlabelled.txt -g {path}/data/breaker_output_unlabelled_gd.txt -b {path}/data/breaker_output_unlabelled_bd.txt {gpt_cmd}

SCIGPT
CKP 0
CKP 1
Loaded ../scigpt/best-checkpoint
CKP 2
100% 2976/2976 [08:11<00:00,  6.06it/s]
src/use_critic.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save(f,np.array(res))
Good Sentences 2499
Bad Sentences 477
Done


gpt Good Sentences 2488
Bad Sentences 472

Scigpt: Good Sentences 2484
Bad Sentences 492

supervised gpt: Good Sentences 2502
Bad Sentences 458

supervised scigpt: Good Sentences 2509
Bad Sentences 467

Extract JSON

In [ ]:
!python src/extract_json.py -t {path}/data/breaker_output_unlabelled_bd.txt -p data/unlabelled_{gpt}/unlabelled_gd_10k.txt -o {path}/data/train_fixer.json

Done


Merge JSONs to Make complete training file

In [ ]:
new_path = 'data/round2_scigpt'

In [ ]:
!mkdir {new_path}

In [ ]:
!python src/merge_files.py -f1 data/round0_synthetic/synthetic_10k.json -f2 {path}/data/train_fixer_A.json -f3 {path}/data/train_fixer.json -o {new_path}/round_train.json

Done
